In [1]:
#=========================================================================================================
# 말뭉치를 가지고, STS Siver dataset 생성하는 예시
#
# - 1단계: 유사문장들 구할 bi_encoder 모델 정의, 스코어를 기록할 cross_encoder 모델 정의
# - 2단계: bi_encoder를 이용하여 입력 말뭉치에서 각 문장당 유사한 문장들 쌍을 구함
# - 3단계: cros_encoder를 이용하여 유사문장들에 대해 각 유사도 스코어를 매김
#=========================================================================================================

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

import math
from sentence_transformers import models, losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('../../')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../../log/s-bert")
device = GPU_info()
seed_everything(111)

logfilepath:../../../log/s-bert_2022-09-26.log
False
device: cpu


In [2]:
# 파라메터들 정의 (필요한 값들 정의)
max_seq_len = 128
top_k = 3             # 훈련데이터에서 몇개의 유사도 문장을 뽑아낼지 정하는 값
train_batch_size = 32

# STS siver_data 만들 말뭉치
corpus_path = '../../../../korpora/moco_1_small.txt'

# STS siver_data  출력 파일 경로
silver_data_file = 'silver_data-moco-sentencebertV2.1.tsv'

# bi_encoder 모델 경로 
# => paraphrase-multilingual-mpnet-base-v2 를 이용
# bi_encoder_path = "../../../model/sbert/teacher/paraphrase-multilingual-mpnet-base-v2/"
bi_encoder_path = "../../../../model/sbert/sbert-mdistilbertV2.1-distil-sts/"

# cross 모델 경로 
ce_model_path = "../../../../model/sbert/sbert-mdistilbertV2.1-distil-sts/"


In [3]:
#======================================================================================================
# bi-encoder 정의
# => 훈련 데이터들의 유사도 문장들를 구할 bi-encoder 모델 정의
#======================================================================================================
word_embedding_encoder = models.Transformer(bi_encoder_path, max_seq_length=max_seq_len)

pooling_encoder = models.Pooling(word_embedding_encoder.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

bi_encoder = SentenceTransformer(modules=[word_embedding_encoder, pooling_encoder])
print(bi_encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [4]:
#======================================================================================================
# cross-encoder 모델 정의
ce_model = CrossEncoder(ce_model_path, num_labels=1)
#======================================================================================================
print(ce_model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ../../../../model/sbert/sbert-mdistilbertV2.1-distil-sts/ and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 문장들을 불러옴.
sentneces = []
with open(corpus_path, encoding="utf-8") as f:
    sentences = [line for line in tqdm(f.read().splitlines()) if (len(line) > 0 and not line.isspace())]

print(f'bi_encoder.encoder Start===>')
# 문장들의 embedding을 구함.(*오래걸림)
embeddings = bi_encoder.encode(sentences, batch_size=train_batch_size, show_progress_bar=True, convert_to_tensor=True)
print(f'bi_encoder.encode End<===')

# 각 문장들에서 유사도가 높은 문장들을 쌍으로 묶음
duplicates = set()
silver_train_data = []

for idx in tqdm(range(len(sentences)), unit="docs"):
    sentence_embedding = embeddings[idx]
    cos_scores = util.pytorch_cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx and (iid, idx) not in duplicates:
            silver_train_data.append((sentences[idx], sentences[iid]))
            duplicates.add((idx, iid))

# 신규 데이터 말뭉치에 대해 score를 추가함
print(f'cross_encoder.predict start===>')
silver_scores = ce_model.predict(silver_train_data, show_progress_bar=True)
print(f'cross_encoder.predict End<===')

print(f'siver_data_len: {len(silver_train_data)}')
print(silver_train_data[0:5])
print(silver_scores[0:5])

  0%|          | 0/3000 [00:00<?, ?it/s]

bi_encoder.encoder Start===>


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

bi_encoder.encode End<===


  0%|          | 0/3000 [00:00<?, ?docs/s]

cross_encoder.predict start===>


Batches:   0%|          | 0/282 [00:00<?, ?it/s]

cross_encoder.predict End<===
9016
[('필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', '외부시스템과 데이터를 주고 받기 위해 사용되는 디렉토리는 다음과 같다.'), ('필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', '특히 당사의 주력사업인 보안관제 및 보안컨설팅 사업의 경우 사업장 내 인력 투입 필요성으로 인한 물리적 요인 및 국가간 규제등 다양한 요인에 따라 해외 진출이 쉽지 않습니다.'), ('필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', '보안관제 및 보안컨설팅 사업의 경우 사업장 내 인력 투입 필요성으로 인한 물리적 요인 및 국가간규제 등 다양한 요인에 따라 해외 진출이 쉽지 않습니다.'), ('외부시스템과 데이터를 주고 받기 위해 사용되는 디렉토리는 다음과 같다.', '유닛플로우에 대한 외부 실행시 사용하는 실행파일.'), ('외부시스템과 데이터를 주고 받기 위해 사용되는 디렉토리는 다음과 같다.', '하위 디렉토리에 해당 런너 관련 파일들이 생성됩니다.')]
[0.50276655 0.48992833 0.48731878 0.48315886 0.48548257]


In [17]:
# silver_data 를 .tsv 파일로 저장 해둠.

with open(silver_data_file, 'w', encoding='utf-8') as f:
    for data, score in tqdm(zip(silver_train_data, silver_scores)):
        f.write(data[0]+'\t')
        f.write(data[1]+'\t')
        f.write(str(score) + '\n')  
        #print(f"({data[0]} : {data[1]}: {str(score)})")

0it [00:00, ?it/s]